In [1]:
import numpy as np
import re
import pandas as pd
import torch
import torch.autograd
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from scipy.stats.mstats import spearmanr
from sklearn.externals import joblib

from pavooc.scoring.feature_extraction import extract_features, split_test_train_valid, normalize_features
from pavooc.scoring.azimuth_dataset import load_dataset
from pavooc.scoring import models
from pavooc.scoring.helper import run_model, run_models, train_predict_n_shuffles
from pavooc.scoring.training import cv_train_test
from pavooc.config import CONSERVATION_FEATURES_FILE, SCALER_FILE


%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
Xdf, Y, gene_position, target_genes = load_dataset()
conservation_scores = pd.read_csv(CONSERVATION_FEATURES_FILE, index_col=0)

combined_features, y, genes, feature_names = extract_features(Xdf, Y, gene_position, conservation_scores, order=1)
normalized_features, scaler = normalize_features(combined_features)
X_train, X_test, y_train, y_test, validation_fold, _ = split_test_train_valid(combined_features, y, joint_scaling=True)

joblib.dump(scaler, SCALER_FILE)

Loaded 149 samples for gene CCDC101 	total number of samples: 149
Loaded 924 samples for gene MED12 	total number of samples: 1073
Loaded 190 samples for gene TADA2B 	total number of samples: 1263
Loaded 109 samples for gene TADA1 	total number of samples: 1372
Loaded 64 samples for gene HPRT1 	total number of samples: 1436
Loaded 154 samples for gene CUL3 	total number of samples: 1590
Loaded 736 samples for gene NF1 	total number of samples: 2326
Loaded 223 samples for gene NF2 	total number of samples: 2549
Loaded 924 samples for gene MED12 	total number of samples: 3473


['/home/moritz/Projects/credit/pavooc/../data/scaler.pkl']

In [17]:

from sklearn.externals import joblib

from pavooc.config import CONSERVATION_FEATURES_FILE, SCALER_FILE
joblib.dump(scaler, SCALER_FILE)

['/home/moritz/Projects/credit/pavooc/../data/scaler.pkl']

In [8]:
Xdf

,,,30mer,Strand
Sequence,Target,drug,,
AAAAAAAACACTGCAACAAG,CD5,nodrug,CAGAAAAAAAAACACTGCAACAAGAGGGTA,sense
AAAAAGCAGCGTCAGTGGAT,CD5,nodrug,TCAGAAAAAGCAGCGTCAGTGGATTGGCCC,sense
AAAACAACGGCCCAGGAGGG,CD5,nodrug,CCAGAAAACAACGGCCCAGGAGGGCGGCCA,sense
AAAAGGAAGATTGCTGATGA,CD45,nodrug,ATACAAAAGGAAGATTGCTGATGAGGGCAG,sense
AAAAGTATCAGTGTGTATAG,THY1,nodrug,CAATAAAAGTATCAGTGTGTATAGAGGTGA,sense
AAACACAAGTGGGAGCAGGC,H2-K,nodrug,CACCAAACACAAGTGGGAGCAGGCTGGTGA,sense
AAACAGTGACGTTCCGTCTC,CD28,nodrug,AACGAAACAGTGACGTTCCGTCTCTGGAAT,sense
AAACAGTGTGCAGTTCCAGT,CD5,nodrug,TGGAAAACAGTGTGCAGTTCCAGTTGGAGG,sense
AAACGCCTAAGCCTAGTTGT,CD45,nodrug,CCAGAAACGCCTAAGCCTAGTTGTGGGGAT,sense


In [3]:
# actually including non-order features as well
order1_features = [not re.match('^[ACTG]{2}(_\d+)?$', feature) for feature in feature_names]
order2_features = [True for feature in feature_names]
# without counts etc..
pure_seq1_features = [bool(re.match('^([ACTG]_\d{1,2})$', feature)) for feature in feature_names]
pure_order1_features = [bool(re.match('^([ACTG]_\d{1,2}|Percent Peptide|Amino Acid Cut position|conservation.*|.*False)$', feature)) for feature in feature_names]
pure_order1_without_conservation_features = [bool(re.match('^([ACTG]_\d{1,2}|Percent Peptide|Amino Acid Cut position|.*False)$', feature)) for feature in feature_names]
order1_without_conservation_features = [not re.match('^([ACTG]{2}(_\d+)?|conservation.*)$', feature) for feature in feature_names]

In [4]:
from pavooc.scoring.models import CNN34

In [10]:
cnn34_results = train_predict_n_shuffles(CNN34, 
                                                    normalized_features,
                                                    order1_features,
                                                    y,
                                                    10,
                                                    0.0003,
                                                    6000)

KeyboardInterrupt: 

In [ ]:
configs = [
    {'model_class': CNN34, 'feature_selector': order1_features, 'loss': nn.MSELoss(), 'learning_rate': 0.0003, 'epochs': 6000},
    ]

results = run_models(X_train, y_train, validation_fold, configs)
model = results[0][2]
predicted_labels = model(torch.from_numpy(X_test)).cpu().data.numpy()

print(spearmanr(predicted_labels, y_test)[0])

In [ ]:
    
configs = [
    {'model_class': CNN34, 'feature_selector': order1_features, 'loss': nn.MSELoss(), 'learning_rate': 0.00034, 'epochs': 6002},
    ]

results = run_models(X_train, y_train, validation_fold, configs)
model = results[0][2]
predicted_labels = model(torch.from_numpy(X_test[:, order1_features])).cpu().data.numpy()
print(max(results[0][1]))
print(spearmanr(predicted_labels, y_test)[0])

1


In [56]:
xxx=model
np.results[0][1]

[0.14953672767542303,
 0.2650470499313614,
 0.3217276550514562,
 0.33831311658932495,
 0.3508942935301225,
 0.36646051636198546,
 0.3836740852749887,
 0.39512237924158866,
 0.40099445490624597,
 0.41228947013063433,
 0.4216424495389706,
 0.42852516680969,
 0.4375982906807524,
 0.4418815746949079,
 0.4457885990958921,
 0.4561056624928287,
 0.461232202652893,
 0.4670785970821433,
 0.4696364375696398,
 0.475407277119418,
 0.4763841654617514,
 0.4800884742995758,
 0.47904533531741794,
 0.47406794211467357,
 0.48671393331374685,
 0.4897066503944287,
 0.49412225790805375,
 0.49966972819870403,
 0.5054237841525172,
 0.5084805580538745,
 0.511505826041232,
 0.5166478629491817,
 0.5203193661908911,
 0.5256366314879337,
 0.527733020415568,
 0.5281062034638453,
 0.5302523889451308,
 0.5431178898834079,
 0.541215303869425,
 0.5460441451052457,
 0.5532296121094104,
 0.558406284671186,
 0.5614233073292159,
 0.5674655674464576,
 0.5692945077324174,
 0.572757412044593,
 0.5777268528445952,
 0.57933550

In [ ]:

configs = [
    {'model_class': CNN34, 'feature_selector': order1_features, 'loss': nn.MSELoss(), 'learning_rate': 0.00029, 'epochs': 6000},
    ]
results = run_models(X_train, y_train, validation_fold, configs)
model = results[0][2]
predicted_labels = model(Variable(torch.from_numpy(X_test))).cpu().data.numpy()

print(spearmanr(predicted_labels, y_test)[0])

In [ ]:
from pavooc.scoring.models import CNN38

In [6]:
cnn38_results = train_predict_n_shuffles(CNN38, 
                                         normalized_features,
                                         order1_features,
                                         y,
                                         9,
                                         0.0003,
                                         6000)

Experiment CNN38_0.0003_6000_MSELoss_1600 already existed. Deleting.


KeyboardInterrupt: 

In [ ]:

configs = [
    {'model_class': CNN38, 'feature_selector': order1_features, 'loss': nn.MSELoss(), 'learning_rate': 0.0003, 'epochs': 15000},
    ]
results = run_models(X_train, y_train, validation_fold, configs)
model = results[0][2]
predicted_labels = model(Variable(torch.from_numpy(X_test[:, order1_features]))).cpu().data.numpy()

print(spearmanr(predicted_labels, y_test)[0])
print(max(results[0][1]))

1
Experiment CNN38_0.0003_15000_MSELoss_160 already existed. Deleting.


In [7]:
print(spearmanr(predicted_labels, y_test)[0])
print(max(results[0][1]))

0.6332038254796858
0.6324884048198914


In [11]:
cv_result = cv_train_test(genes, normalized_features[:, order1_features], y, CNN38, 0.0003, 3000)

Experiment CNN38_0.0003_3000_MSELoss_cv|0 already existed. Deleting.


/usr/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/usr/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [12]:
rand_shuffle_results = train_predict_n_shuffles(CNN38, 
                                                normalized_features,
                                                order1_features,
                                                y,
                                                10,
                                                0.0003,
                                                6000)

Experiment CNN38_0.0003_6000_MSELoss_1600 already existed. Deleting.
Experiment CNN38_0.0003_6000_MSELoss_1601 already existed. Deleting.


KeyboardInterrupt: 